In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Data Collection and Pre-Processing

In [ ]:
# loading the data from the csv file to apandas dataframe
movies_data = pd.read_csv('/content/drive/MyDrive/Dataset/movies.csv')

In [ ]:
# printing the first 5 rows of the dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
# number of rows and columns in the data frame

movies_data.shape

(4803, 24)

In [ ]:
# selecting the relevant features for recommendation

selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [ ]:
# checking missing values in each feature
#for features in selected_features:
 #   print(movies_data[features].isnull().sum())

In [ ]:
# replacing the null valuess with null string

for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [ ]:
# combining all the 5 selected features

combined_features = movies_data['genres']+' '+movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+movies_data['director']

In [ ]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


In [ ]:
combined_features[0]

'Action Adventure Fantasy Science Fiction culture clash future space war space colony society Enter the World of Pandora. Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez James Cameron'

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
corpus=[]
for i in range(len(combined_features)):
  review= re.sub('[^a-zA-Z]', ' ', combined_features[i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus[0]

'action adventur fantasi scienc fiction cultur clash futur space war space coloni societi enter world pandora sam worthington zoe saldana sigourney weaver stephen lang michel rodriguez jame cameron'

In [ ]:
from numpy.lib.function_base import vectorize
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()

In [ ]:
feature_vectors = vectorizer.fit_transform(corpus)

In [ ]:
print(feature_vectors)

  (0, 2071)	0.17552313506858697
  (0, 6793)	0.11463157223101918
  (0, 11596)	0.19738383622085057
  (0, 9066)	0.15239002415366618
  (0, 7728)	0.2052522310553095
  (0, 13033)	0.15213516971481178
  (0, 14587)	0.20164827377007877
  (0, 12549)	0.20929853505545373
  (0, 11874)	0.22127568137478468
  (0, 15139)	0.2052522310553095
  (0, 14871)	0.24026470377934847
  (0, 11900)	0.15264686191548815
  (0, 10240)	0.27652273888326423
  (0, 14863)	0.12875501676155404
  (0, 4305)	0.24415195452788352
  (0, 12728)	0.21738843062624963
  (0, 2782)	0.24855895595249378
  (0, 14523)	0.12740774864377094
  (0, 12829)	0.34513124291779446
  (0, 5083)	0.16734367774450565
  (0, 2641)	0.22568268279939493
  (0, 3161)	0.21560402385508065
  (0, 4735)	0.10471198725789553
  (0, 12131)	0.1053209249350964
  (0, 4580)	0.11273156644207886
  :	:
  (4801, 15115)	0.29620361121379385
  (4801, 4209)	0.2536402429374028
  (4801, 3016)	0.271699171736498
  (4801, 240)	0.1819402672747889
  (4801, 6077)	0.29620361121379385
  (4801, 103

Cosine Similarity

In [ ]:
# getting the similarity scores using cosine similarity

similarity = cosine_similarity(feature_vectors)

In [ ]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [ ]:
print(similarity.shape)

(4803, 4803)


Getting the movie name from the user

In [ ]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : Iron man


In [ ]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [ ]:
# finding the close match for the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [ ]:
close_match = find_close_match[0]
print(close_match)

Iron Man


In [ ]:
# finding the index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

68


In [ ]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)

[(0, 0.036279148864686264), (1, 0.061558683705088574), (2, 0.016092132930621703), (3, 0.04036790530962703), (4, 0.03670710172252938), (5, 0.014984720198289544), (6, 0.04056023641993075), (7, 0.2559459585117725), (8, 0.00862439867233422), (9, 0.10665875073850486), (10, 0.11445692316127394), (11, 0.014425658558468866), (12, 0.014745006244598808), (13, 0.013744197112720027), (14, 0.1344353775963146), (15, 0.01926500467194919), (16, 0.24471600783867883), (17, 0.014469193298354423), (18, 0.03927896524269902), (19, 0.08522031330525329), (20, 0.0833569530723637), (21, 0.012966856022499917), (22, 0.007600482681465084), (23, 0.007770423866269052), (24, 0.013990230591202722), (25, 0.0), (26, 0.23252314013567685), (27, 0.03364715474633025), (28, 0.06609997435292247), (29, 0.015891760262266477), (30, 0.1142378515579299), (31, 0.3467917032540425), (32, 0.01894997570690099), (33, 0.1365048048461158), (34, 0.0), (35, 0.03827407061606146), (36, 0.03609109060474577), (37, 0.008935570647780906), (38, 0.

In [ ]:
len(similarity_score)

In [ ]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 
print(sorted_similar_movies)

[(68, 1.0000000000000002), (79, 0.49707922629926665), (31, 0.3467917032540425), (7, 0.2559459585117725), (16, 0.24471600783867883), (26, 0.23252314013567685), (85, 0.20455741986311912), (182, 0.1953649618313301), (511, 0.17533432290975318), (3623, 0.17341034843112504), (203, 0.1681308702851011), (64, 0.16563790478583978), (788, 0.1646084646483868), (174, 0.1608301763898909), (101, 0.15957134204907103), (4401, 0.15440968284752232), (1740, 0.1464232516619329), (126, 0.1416030302429484), (33, 0.1365048048461158), (46, 0.13603580281799374), (14, 0.1344353775963146), (2487, 0.13414409901343505), (783, 0.13341389571369588), (169, 0.1323008099198056), (870, 0.1318121155980149), (138, 0.1301591654273605), (2442, 0.1289905820697534), (94, 0.12788866195702805), (2651, 0.1278591040031402), (661, 0.12614513974414274), (1192, 0.12550061387481526), (607, 0.12455949094993159), (131, 0.12344449967597909), (38, 0.12023427123751528), (1553, 0.11849950588900998), (1368, 0.11459406557071378), (10, 0.11445

In [ ]:
# print the name of similar movies based on the index

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . The Avengers
6 . Captain America: Civil War
7 . Captain America: The Winter Soldier
8 . Ant-Man
9 . X-Men
10 . Made
11 . X2
12 . X-Men: Apocalypse
13 . Deadpool
14 . The Incredible Hulk
15 . X-Men: First Class
16 . The Helix... Loaded
17 . Kick-Ass 2
18 . Thor: The Dark World
19 . X-Men: The Last Stand
20 . X-Men: Days of Future Past
21 . Man of Steel
22 . Duets
23 . Mortdecai
24 . Captain America: The First Avenger
25 . Superman II
26 . The Last Airbender
27 . Southland Tales
28 . Guardians of the Galaxy
29 . The Good Night


Movie Recommendation Sytem

In [ ]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : Iron Man
Movies suggested for you : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . The Avengers
6 . Captain America: Civil War
7 . Captain America: The Winter Soldier
8 . Ant-Man
9 . X-Men
10 . Made
11 . X2
12 . X-Men: Apocalypse
13 . Deadpool
14 . The Incredible Hulk
15 . X-Men: First Class
16 . The Helix... Loaded
17 . Kick-Ass 2
18 . Thor: The Dark World
19 . X-Men: The Last Stand
20 . X-Men: Days of Future Past
21 . Man of Steel
22 . Duets
23 . Mortdecai
24 . Captain America: The First Avenger
25 . Superman II
26 . The Last Airbender
27 . Southland Tales
28 . Guardians of the Galaxy
29 . The Good Night
